In [1]:
# constant
DF_PATH = "../data/processed/1_preprocessed_df.pkl"
DF_EXPORT = "../data/processed/04_bow_cv_results_df.pkl"

In [2]:

import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import re

# preprocess
from sklearn.compose import make_column_transformer , ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso , LassoCV , Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor



from sklearn.model_selection import cross_val_score


# metric
from sklearn.metrics import make_scorer,mean_squared_error
scoring = make_scorer(mean_squared_error) 

# pipeline
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
from autocorrect import Speller
spell = Speller()
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin


In [3]:
# Read Data 
df = pd.read_pickle(DF_PATH)
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
1,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
2,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
3,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Napa,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature
4,US,"Slightly reduced, this wine offers a chalky, t...",NaN,87,34.0,California,Alexander Valley,Sonoma,Virginie Boone,@vboone,Louis M. Martini 2012 Cabernet Sauvignon (Alex...,Cabernet Sauvignon,Louis M. Martini


 ## use the text features
* a text-based model using a bag-of-words approach
> only work in the description feature as text feature, may be use more of text features in future.
#### BOW

##### Sample to test and tuning

In [5]:
sample = df.sample(6000) # small sample to avoid much time of computing
X_train,X_test,y_train,y_test = train_test_split(sample['description'] , sample['points'] ,  random_state=42)

### preprocessing:
   - clean text
   - drop stop words
   - correct the spelling
   - root of the word
- convert the function to column transformer ( to play with Pipelines )
   

In [6]:
TEXT_CLEANING_RE = "[^A-Za-z0-9]"
stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

# for every row
def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(spell(token)))
            else:
                tokens.append(token)
    return " ".join(tokens)


# for the DataFrame
def _pre_all(data,stem = True):
    X = data.apply(lambda x : preprocess(x))
  
    return X
    
pre_all = FunctionTransformer(_pre_all)

In [7]:
# test a model with the default hyperparametrs
model = make_pipeline(pre_all,CountVectorizer(lowercase=False,analyzer = 'word'),RandomForestRegressor())
score = cross_val_score(model, X_train,y_train, scoring = scoring)
score

KeyboardInterrupt: 

cross val score to test the sample, it is stable and we can do tuning with it or not 

In [8]:
model.fit(X_train,y_train)

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function _pre_all at 0x00000211563E0040>)),
                ('countvectorizer', CountVectorizer(lowercase=False)),
                ('randomforestregressor', RandomForestRegressor())])

In [10]:
print(mean_squared_error(y_test,model.predict(X_test)))

5.5946744


> looks good what about if we tuning 

### Hyperparameter tuning
### BOW + tuning

In [7]:
# base pipline
pipe = Pipeline([
    ('preprocess',pre_all),
    ('vectorizer',CountVectorizer()),
    ('vec2' , TfidfTransformer() ),
     ('regressor', Lasso())
])


param_grid = [{'regressor': [RandomForestRegressor()],
               'regressor__ccp_alpha': np.linspace(0.0, 0.0030, 5),
              # 'regressor__max_iter' : [500,600],
               'vectorizer': [CountVectorizer()],
               'vec2' : [TfidfTransformer() , 'passthrough'],
              'vectorizer__analyzer' : ['word'],
              'vectorizer__ngram_range' : [(1, 1),(2, 2)]
              }]

grid = GridSearchCV(pipe, param_grid ,n_jobs=-2 , scoring = scoring , cv = 3)
#with config_context(target_offload="gpu:0"):
grid.fit(X_train,y_train)
print(' mean_squared_error on train set = ', grid.score(X_train, y_train))
print(' mean_squared_error on test set = ', grid.score(X_test, y_test))

In [ ]:
print(grid.best_params_)

{'regressor': Lasso(alpha=0.0001), 'regressor__alpha': 0.0001, 'vec2': TfidfTransformer(), 'vectorizer': CountVectorizer(), 'vectorizer__analyzer': 'word', 'vectorizer__ngram_range': (2, 2)}


In [119]:
results = pd.DataFrame(grid.cv_results_)


In [ ]:
results.to_pickle(EXPORT_PATH)

In [120]:
results.sort_values(by=['mean_test_score'],ascending=True).reset_index().loc[0,'params']

{'regressor': Ridge(),
 'regressor__alpha': 0.0001,
 'vec2': 'passthrough',
 'vectorizer': CountVectorizer(ngram_range=(2, 2)),
 'vectorizer__analyzer': 'word',
 'vectorizer__ngram_range': (1, 1)}

In [121]:
params = results.sort_values(by=['mean_test_score'],ascending=True).reset_index().loc[0,'params'].copy()
params

{'regressor': Ridge(),
 'regressor__alpha': 0.0001,
 'vec2': 'passthrough',
 'vectorizer': CountVectorizer(ngram_range=(2, 2)),
 'vectorizer__analyzer': 'word',
 'vectorizer__ngram_range': (1, 1)}

##### params to use in the comming BOW models